In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import sklearn.metrics as metrics
from scipy.stats import randint, uniform
import pickle

In [3]:
def zscore_normalization(df, column_name):
    mean = df[column_name].mean()
    sd = df[column_name].std()
    df[column_name] = (df[column_name] - mean) / sd

def preprocess(df):
    mapping = {'legitimate': 0, 'phishing': 1}
    df['status'] = df['status'].map(mapping)
    df = df.drop(columns=['url'])
    for col in df.columns:
        if col != 'status':
            zscore_normalization(df, col)
    corr_matrix = df.corr(numeric_only=True)
    target_corr = corr_matrix['status']
    threshold = 0.1
    relevant_features = target_corr[abs(target_corr) > threshold].index.tolist()
    df = df[relevant_features]

    return df

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/tsimhadri-ews/internproject/sashok-branch/Data/dataset_B_05_2020.csv')

In [5]:
preprocess(df)

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_eq,nb_slash,...,safe_anchor,empty_title,domain_in_title,domain_with_copyright,domain_registration_length,domain_age,dns_record,google_index,page_rank,status
0,-0.436308,-0.193955,-0.421002,0.379100,-0.477963,-0.142908,-0.387447,-0.197595,-0.293670,-0.685131,...,-0.948572,-0.377532,-1.860391,1.129145,-0.549275,-1.307537,6.977922,0.934223,0.320960,0
1,0.287054,0.177200,2.375078,-1.081089,-0.477963,-0.142908,-0.387447,-0.197595,-0.293670,0.377426,...,1.610715,-0.377532,0.537474,-0.885548,-0.510000,0.548447,-0.143297,0.934223,-0.467387,1
2,1.173173,2.682495,2.375078,1.109194,0.001174,-0.142908,2.356370,2.237458,2.711387,0.377426,...,1.610715,-0.377532,0.537474,-0.885548,-0.587322,-0.018838,-0.143297,0.934223,-1.255733,1
3,-0.779906,-0.936265,-0.421002,-0.350995,-0.477963,-0.142908,-0.387447,-0.197595,-0.293670,-1.216410,...,0.650982,-0.377532,0.537474,-0.885548,-0.528410,-1.307537,-0.143297,-1.070314,-0.073213,0
4,-0.110795,-0.565110,-0.421002,-0.350995,0.480311,-0.142908,-0.387447,-0.197595,-0.293670,0.377426,...,-0.948572,-0.377532,-1.860391,1.129145,-0.329581,1.323276,-0.143297,-1.070314,1.109306,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,-0.291636,-0.379533,-0.421002,-0.350995,-0.477963,-0.142908,-0.387447,-0.197595,-0.293670,-0.153852,...,-0.948572,-0.377532,-1.860391,-0.885548,-0.054656,0.429070,-0.143297,-1.070314,1.109306,0
11426,0.413642,-0.286744,-0.421002,1.839288,-0.477963,6.287966,2.356370,-0.197595,0.708016,0.377426,...,-0.948572,-0.377532,0.537474,-0.885548,-0.345536,0.857671,-0.143297,0.934223,-1.255733,1
11427,0.793408,-0.472321,2.375078,-0.350995,2.396858,-0.142908,2.356370,-0.197595,0.708016,0.377426,...,1.098857,-0.377532,-1.860391,-0.885548,2.843096,1.432679,-0.143297,0.934223,2.685999,0
11428,-0.418224,0.826721,-0.421002,-0.350995,-0.477963,-0.142908,-0.387447,-0.197595,-0.293670,-0.685131,...,-0.948572,-0.377532,0.537474,-0.885548,-0.500181,-0.394668,-0.143297,-1.070314,0.320960,0
